In [10]:
import torch
import numpy as np

In [11]:
w = torch.tensor(2.8, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

x = torch.tensor([1.0, 2.0])

In [12]:
y_pred = w * x + b
print(f"y_pred: {y_pred}")

y_pred: tensor([3.8000, 6.6000], grad_fn=<AddBackward0>)


In [13]:
y_true = torch.tensor([10.0, 10.0])
loss = ((y_true - y_pred) ** 2).mean()
print(f"loss: {loss.item()}")

loss: 25.0


In [14]:
print(f"w before backward: w.grad = {w.grad}")

w before backward: w.grad = None


In [15]:
loss.backward()


In [16]:
print(f"apres backward: w.grad = {w.grad}")
print(f"apres backward: b.grad = {b.grad}")

apres backward: w.grad = -13.0
apres backward: b.grad = -9.600000381469727


In [17]:
learning_rate = 0.01
with torch.no_grad():
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad


In [18]:
print(f"nouveau poids: w = {w}, b = {b}")

nouveau poids: w = 2.929999828338623, b = 1.0959999561309814


In [19]:
import torch.nn as nn
perception = nn.Linear(1, 1)

print(f"poids: {perception.weight}")
print(f"biais: {perception.bias}")

poids: Parameter containing:
tensor([[-0.1041]], requires_grad=True)
biais: Parameter containing:
tensor([-0.9632], requires_grad=True)


In [20]:
# Créer un tensor de la forme (batch_size, in_features)
# perception attend des entrées de forme (N, 1) car nn.Linear(1,1)
x = torch.tensor([[3.0]])  # batch de 2 échantillons, 1 feature chacun
print(f"input: {x}")
y_prod = perception(x)
print(y_prod)

input: tensor([[3.]])
tensor([[-1.2756]], grad_fn=<AddmmBackward0>)


In [21]:
y_true = torch.tensor([[10.0]])
criterium = nn.MSELoss()
loss = criterium(y_prod, y_true)
print(f"loss: {loss.item()}")

loss: 127.14002227783203


In [22]:
loss.backward()

In [23]:
print(f"gradient du poids: {perception.weight.grad}")
print(f"gradient du biais: {perception.bias.grad}")

gradient du poids: tensor([[-67.6538]])
gradient du biais: tensor([-22.5513])


In [24]:
import torch.optim as optim
optimizer = optim.SGD(perception.parameters(), lr=0.01)


In [25]:
optimizer.step()


In [26]:
print(f"nouveau poids : {perception.weight} {perception.bias}")

nouveau poids : Parameter containing:
tensor([[0.5724]], requires_grad=True) Parameter containing:
tensor([-0.7377], requires_grad=True)


In [27]:
optimizer.zero_grad()

In [28]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [29]:
model = SimpleNet()

In [30]:
model

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [31]:
# Entrée : un batch d'un seul échantillon avec 2 features (SimpleNet attend 2 features)
x = torch.tensor([[1.0, 2.0]])  # forme (1, 2)
y = torch.tensor([[5.0]])  # forme (1, 1)
y_pred = model(x)
print(y_pred)

tensor([[0.5712]], grad_fn=<SigmoidBackward0>)


In [32]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose
import matplotlib.pyplot as plt


In [33]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [34]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [35]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [36]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.fc(x)
        return logits


In [37]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [38]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)

In [39]:
logits

tensor([[-0.1238, -0.0310, -0.0907,  0.0179,  0.0011, -0.1579, -0.0785,  0.0424,
         -0.1041, -0.0794]], grad_fn=<AddmmBackward0>)

In [40]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0937, 0.1028, 0.0968, 0.1079, 0.1061, 0.0905, 0.0980, 0.1106, 0.0955,
         0.0979]], grad_fn=<SoftmaxBackward0>)

In [41]:
y_pred = pred_probab.argmax(1)
y_pred

tensor([7])

In [42]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [43]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader):
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        # Calcul des prédictions et de la perte
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [44]:
def test(dataloader, model, loss_fn):
    # Si on passe un dataset directement, on le transforme en DataLoader
    if isinstance(dataloader, torch.utils.data.Dataset):
        dataloader = DataLoader(dataloader, batch_size=64)

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0.0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = 100.0 * correct / size
    print(f"Test set: Accuracy: {accuracy:0.1f}%, Avg loss: {test_loss:>8f}")


In [45]:
epoche = 10
for t in range(epoche):
    print(f"epoche {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)
print("Done!")

epoche 1
-------------------------------


c:\laragon\www\A4\5-ai\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss: 2.306401  [   64/60000]
loss: 2.301030  [ 6464/60000]
loss: 2.291095  [12864/60000]
loss: 2.291959  [19264/60000]
loss: 2.289494  [25664/60000]
loss: 2.268740  [32064/60000]
loss: 2.287798  [38464/60000]
loss: 2.268719  [44864/60000]
loss: 2.267082  [51264/60000]
loss: 2.270978  [57664/60000]
Test set: Accuracy: 21.5%, Avg loss: 2.267896
epoche 2
-------------------------------
loss: 2.266053  [   64/60000]
loss: 2.269327  [ 6464/60000]
loss: 2.266765  [12864/60000]
loss: 2.264189  [19264/60000]
loss: 2.243588  [25664/60000]
loss: 2.251380  [32064/60000]
loss: 2.243763  [38464/60000]
loss: 2.218952  [44864/60000]
loss: 2.232290  [51264/60000]
loss: 2.215972  [57664/60000]
Test set: Accuracy: 54.6%, Avg loss: 2.218692
epoche 3
-------------------------------
loss: 2.234583  [   64/60000]
loss: 2.200405  [ 6464/60000]
loss: 2.213058  [12864/60000]
loss: 2.206425  [19264/60000]
loss: 2.157280  [25664/60000]
loss: 2.177636  [32064/60000]
loss: 2.158101  [38464/60000]
loss: 2.163326  

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms


class ImageClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [47]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) 
])

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [48]:
model = ImageClassifierModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [49]:
print("Starting Model Training...")
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print("Finished Training.")

Starting Model Training...
[Epoch 1, Batch   100] loss: 1.767
[Epoch 1, Batch   200] loss: 0.317
[Epoch 1, Batch   300] loss: 0.196
[Epoch 1, Batch   400] loss: 0.143
[Epoch 1, Batch   500] loss: 0.122
[Epoch 1, Batch   600] loss: 0.123
[Epoch 1, Batch   700] loss: 0.104
[Epoch 1, Batch   800] loss: 0.088
[Epoch 1, Batch   900] loss: 0.082
[Epoch 2, Batch   100] loss: 0.067
[Epoch 2, Batch   200] loss: 0.066
[Epoch 2, Batch   300] loss: 0.056
[Epoch 2, Batch   400] loss: 0.064
[Epoch 2, Batch   500] loss: 0.072
[Epoch 2, Batch   600] loss: 0.074
[Epoch 2, Batch   700] loss: 0.054
[Epoch 2, Batch   800] loss: 0.066
[Epoch 2, Batch   900] loss: 0.063
[Epoch 3, Batch   100] loss: 0.038
[Epoch 3, Batch   200] loss: 0.060
[Epoch 3, Batch   300] loss: 0.043
[Epoch 3, Batch   400] loss: 0.044
[Epoch 3, Batch   500] loss: 0.045
[Epoch 3, Batch   600] loss: 0.049
[Epoch 3, Batch   700] loss: 0.047
[Epoch 3, Batch   800] loss: 0.040
[Epoch 3, Batch   900] loss: 0.042
[Epoch 4, Batch   100] loss:

In [50]:
print("Exporting TRAINED model to ONNX...")
model.eval()

example_inputs = torch.randn(1, 1, 28, 28) 

torch.onnx.export(
    model, 
    example_inputs, 
    f="image_classifier_model.onnx",
    input_names=['input'], 
    output_names=['output'],
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    dynamo=False
)

print("\n--- SUCCESS ---")

Exporting TRAINED model to ONNX...


C:\Users\Laerthe\AppData\Local\Temp\ipykernel_6716\4041265909.py:6: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(



--- SUCCESS ---


In [130]:
# CNN
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [131]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=4, shuffle=True, num_workers=2
)

testset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=4, shuffle=False, num_workers=2
)

classes = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [132]:
writer = SummaryWriter('runs/mnist_cnn')

In [133]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=0)
        
        self.dropout = nn.Dropout(p=0.5) 
        
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        
        x = self.dropout(F.relu(self.fc1(x))) 
        
        x = self.fc2(x)
        return x

net = CNN()

In [134]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [135]:
# Get one batch of training data
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Log the model graph
writer.add_graph(net, images)

In [136]:
for epoch in range(5):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            global_step = epoch * len(trainloader) + i
            writer.add_scalar('Training Loss/Batch', running_loss / 2000, global_step)
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

writer.close()
print('Finished Training')

[1,  2000] loss: 0.944
[1,  4000] loss: 0.277
[1,  6000] loss: 0.212
[1,  8000] loss: 0.170
[1, 10000] loss: 0.144
[1, 12000] loss: 0.127
[1, 14000] loss: 0.119
[2,  2000] loss: 0.111
[2,  4000] loss: 0.100
[2,  6000] loss: 0.089
[2,  8000] loss: 0.087
[2, 10000] loss: 0.089
[2, 12000] loss: 0.082
[2, 14000] loss: 0.076
[3,  2000] loss: 0.062
[3,  4000] loss: 0.070
[3,  6000] loss: 0.064
[3,  8000] loss: 0.075
[3, 10000] loss: 0.067
[3, 12000] loss: 0.064
[3, 14000] loss: 0.071
[4,  2000] loss: 0.060
[4,  4000] loss: 0.055
[4,  6000] loss: 0.056
[4,  8000] loss: 0.056
[4, 10000] loss: 0.063
[4, 12000] loss: 0.057
[4, 14000] loss: 0.049
[5,  2000] loss: 0.046
[5,  4000] loss: 0.054
[5,  6000] loss: 0.047
[5,  8000] loss: 0.047
[5, 10000] loss: 0.052
[5, 12000] loss: 0.049
[5, 14000] loss: 0.047
Finished Training


In [137]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [138]:
print("Exporting TRAINED model to ONNX...")
model.eval()

example_inputs = torch.randn(1, 1, 28, 28) 

torch.onnx.export(
    model, 
    example_inputs, 
    f="image_classifier_model.onnx",
    input_names=['input'], 
    output_names=['output'],
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    dynamo=False
)

print("\n--- SUCCESS ---")

Exporting TRAINED model to ONNX...

--- SUCCESS ---


C:\Users\Laerthe\AppData\Local\Temp\ipykernel_6716\4041265909.py:6: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
